In [1]:
import sys
import os
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

# Avoids warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import tensorflow as tf
sys.path.append('../../../../../src/')
import ScalableLib.classifier.Multiband as multiband


In [2]:
tf.__version__

'2.16.1'

Classifier mode only should ignore physical parameters listed in the input.

In [3]:
# # To see if the system regognises the GPU
# device = 0
# devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.set_visible_devices(devices[device], 'GPU')
# tf.config.experimental.set_memory_growth(device=devices[device], enable=True)

# device_name = tf.config.experimental.get_device_details(devices[device])['device_name']
# print("Using {}".format(device_name))

Find the different folds and train a model using the stored data.

In [4]:
survey = 'PanStarrs'
path = os.path.join('../02_CreateRecords/', survey, 'Folds/Fold_*',)
folds = glob(path)
folds.sort()
folds

['../02_CreateRecords/PanStarrs/Folds/Fold_1',
 '../02_CreateRecords/PanStarrs/Folds/Fold_2',
 '../02_CreateRecords/PanStarrs/Folds/Fold_3',
 '../02_CreateRecords/PanStarrs/Folds/Fold_4',
 '../02_CreateRecords/PanStarrs/Folds/Fold_5',
 '../02_CreateRecords/PanStarrs/Folds/Fold_6',
 '../02_CreateRecords/PanStarrs/Folds/Fold_7']

Create folder results

In [5]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')

Define the arguments for all the models.

In [6]:
train_args = {
            'hidden_size_bands':[64, 64, 64],
            'hidden_size_central':[64, 64, 64],
            'fc_layers_bands':[128,128],
            'fc_layers_central':[128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':1000,
            'num_threads':7,
            'batch_size':512,
            'dropout':0.35,
            'lr':[[5e-3]*5, 2.5e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':500, 
            'use_class_weights':False,# Not Used
            'mode' : 'classifier+regression',
            }
# loss_weights = {'Class':300.0, 'T_eff':1.0,'Radius':1e0}
loss_weights = {'Class':300.0, 'T_eff':20.0,'Radius':1e0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': ['T_eff', 'Radius'],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,
                    'N_skip' : 2, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': False,
                    'train_steps_central' : 1,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args    
                    }



In [7]:
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'

    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../02_CreateRecords/'+survey+'/', './').replace('/Folds/', '/Results/')
    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)
        
    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = base_dir+'metadata_preprocess.json'
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers')
    # Define the train args
    train_args = {**train_args, **train_args_specific}


    train_files = base_dir+'train/*.tfrecord'
    val_files = base_dir+'val/*.tfrecord'
    test_files = base_dir+'test/*.tfrecord'

    new = multiband.Network()
    new.train(train_args, train_files, val_files, test_files)
    # new.train_loop()
    break

./Results/Fold_1/Models/20241204-0039


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:1210: UserWarning: Layer 'rnn_layers_bands' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encoutered: ''missing a required argument: 'sequences'''
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:359: UserWarning: `build()` was called on layer 'rnn_layers_bands', however the layer does not have 

TypeError: Exception encountered when calling RNNLayersBands.call().

[1mmissing a required argument: 'sequences'[0m

Arguments received by RNNLayersBands.call():
  • args=('<KerasTensor shape=(None, None, 4), dtype=float32, sparse=None, name=input_LC_0>', '<KerasTensor shape=(None,), dtype=int32, sparse=None, name=N_0>')
  • kwargs=<class 'inspect._empty'>

In [13]:
import numpy as np
for batch in new.dataset_test:
    ids = batch[0]['ID'].numpy().astype(np.int64)
    teff = batch[1]['T_eff'].numpy()
    radius = batch[1]['Radius'].numpy()
    # df = pd.DataFrame(dsimpata = np.array())
    break

In [23]:
df = pd.DataFrame(data = {'T_eff':teff, 'Radius':radius}, index=ids)
df_teff = df[df.T_eff>0].drop('Radius', axis=1)
df_rad = df[df.Radius>0].drop('T_eff', axis=1)

In [24]:
df_teff

,T_eff
84300158481827892,1.006477
74163190788076994,0.528689
103253008374854869,0.755203
74022884304620926,0.942571
131542740608734400,0.627559
...,...
73262728454981330,0.157801
72963155537180060,0.881453
179912518832710896,0.700164
109452465097187253,0.721828
